### GWU STAT 4197/6197
##### Week 6 SAS Code Examples: Combining SAS Data Sets
(Source: SAS Documentation for Code Explanation)

In [1]:
*Ex1_concat_interleave.sas ;
ods html close;
options nocenter nonumber nodate nosource nonotes;
DATA D1;
  INPUT CustID Month $ purchased_amount;
  DATALINES; 
  11 Jan 237.4 
  12 Jan 249.2 
  13 Jan 227.7 
  ;


SAS Connection established. Subprocess id is 11816



In [2]:
ods html close;
options nocenter nonumber nodate nosource nonotes;
DATA D2;
 INPUT CustID Month $ purchased_amount;
 DATALINES;
 11 Feb 288.2 
 12 Feb 221.7 
 13 Feb 274.4  
 14 Feb 222.9
;


In [9]:
 DATA concat; 
   SET D1 D2; 
 run;
title 'Use the SET statement to concatenate data sets';
PROC PRINT DATA=concat noobs ; 
run;

CustID,Month,purchased_amount
11,Jan,237.4
12,Jan,249.2
13,Jan,227.7
11,Feb,288.2
12,Feb,221.7
13,Feb,274.4
14,Feb,222.9


In [10]:
*Ex1_concat_interleave.sas;
options nocenter nonumber nodate;
Data master; set D1; run;
Data add; set D2; run;
proc append base=master  data=add;
run;
title1 'Use the PROC APPEND to concatenate data sets';
proc print data=master noobs;
run;


CustID,Month,purchased_amount
11,Jan,237.4
12,Jan,249.2
13,Jan,227.7
11,Feb,288.2
12,Feb,221.7
13,Feb,274.4
14,Feb,222.9


In [11]:
*Ex1_concat_interleave.sas (Part 3);
options nocenter nonumber nodate;

proc sql;
 create table concat_sql as
 select * from D1
   union 
 select * from D2
 order by Month desc;
 title1 'Vertical Joining Using PROC SQL';
 select * from concat_sql;
quit;

CustID,Month,purchased_amount
11,Jan,237.4
12,Jan,249.2
13,Jan,227.7
11,Feb,288.2
12,Feb,221.7
13,Feb,274.4
14,Feb,222.9


In [12]:

*Ex1_concat_interleave.sas (Part 4);
options nocenter nonumber nodate;
*Interleave Data Sets D1 and D2;
PROC SORT DATA=D1 out=D1sorted; 
  BY CustID Month; 
run;
PROC SORT DATA=D2 out=D2sorted; 
BY CustID Month; run;
DATA interleave;
SET D1sorted D2sorted ; 
 BY CustID descending Month;
run;

PROC PRINT DATA=interleave noobs;
title1 'Data - Interleaved';
run;

CustID,Month,purchased_amount
11,Jan,237.4
11,Feb,288.2
12,Jan,249.2
12,Feb,221.7
13,Jan,227.7
13,Feb,274.4
14,Feb,222.9


With the UNION operator with PROC SQL, rows from intermediate
result sets are concatenated. 

The default behavior of the UNION operator
is that the duplicate rows are removed from the final results. 



In [13]:
*Ex1_concat_interleave.sas (Part 5);
options nocenter nonumber nodate;
proc sql;
 create table concat_sql_i as
 select * from D1
   union 
 select * from D2
 order by CustID, Month desc;
  title1
 'Vertical Joining Using PROC SQL /Interleaved';
 select * from concat_sql_i;
quit;

CustID,Month,purchased_amount
11,Jan,237.4
11,Feb,288.2
12,Jan,249.2
12,Feb,221.7
13,Jan,227.7
13,Feb,274.4
14,Feb,222.9


In [12]:
*Ex1_concat_interleave.sas (Part 6);
*** Create a data set with variables that have the same attribute 
    as those in an existing SAS data set- code idea from
    Marths Messineo (2017);
options nocenter nonumber nodate;
options nocenter nonumber nodate;
  data class1 ;
   set sashelp.class;
  run; 
  
data class2;
  if (0) then set SASHELP.CLASS;
  input name sex age height weight;
  datalines;
  Kia F 13 62  102
  ; 
proc append base=class1 data=class2;
run;
title1 'Appending data sets (PROC APPEND)';
proc print data=class1; 
run;

Obs,Name,Sex,Age,Height,Weight
1,Alfred,M,14,69.0,112.5
2,Alice,F,13,56.5,84.0
3,Barbara,F,13,65.3,98.0
4,Carol,F,14,62.8,102.5
5,Henry,M,14,63.5,102.5
6,James,M,12,57.3,83.0
7,Jane,F,12,59.8,84.5
8,Janet,F,15,62.5,112.5
9,Jeffrey,M,13,62.5,84.0
10,John,M,12,59.0,99.5


In [3]:
*Ex2_match_merge_sql_outer.sas (Part 1);
options nocenter nodate nonumber;
DATA BIRTH;
  INPUT id $ dob : mmddyy.;
  FORMAT dob  mmddyy10.;
  DATALINES; 
03 03/31/1944 
04 08/11/1950
01 01/09/1954 
02 09/12/1959 
05 07/18/1941
;
PROC SORT data=BIRTH; by id; 
title1 'BIRTH File - Listing'; 
PROC PRINT data=BIRTH noobs;  run;


id,dob
01,01/09/1954
02,09/12/1959
03,03/31/1944
04,08/11/1950
05,07/18/1941


In [4]:
DATA DEATH;
input id $ dod : mmddyy.;
FORMAT dod mmddyy10.;
DATALINES;
07 12/31/2011 
08 02/14/2012
04 12/31/2010 
05 12/12/2012 
06 12/29/2011 
; 
PROC SORT data=DEATH; by id; 
title1 'DEATH File - Listing'; footnote;
PROC PRINT data=DEATH noobs;  run;

id,dod
04,12/31/2010
05,12/12/2012
06,12/29/2011
07,12/31/2011
08,02/14/2012


### Merging
* The MERGE statement joins observations from two or more SAS data sets into single observations.
* The BY statement specifies the common variables to match-merge observations.
* The variables in the BY statement must be common to all data sets.
* The data sets listed in the MERGE statement must be sorted in the order of the values of the variables that are listed in the BY statement, or they must have an appropriate index.
* Variable name, type, and length attributes are established by the first data set.
* In one-to-one or one to many merge, variable values might come from the last data set.

In [16]:
*Ex2_match_merge_sql_outer.sas;
options nocenter nodate nonumber;
** DATA Step Merge (match-merge);
data match_merge;
 merge  BIRTH DEATH ; 
 by id;
 run;
title1 'DATA Step Merge (Match-Merge)';
proc print data=match_merge noobs;
run;

id,dob,dod
01,01/09/1954,.
02,09/12/1959,.
03,03/31/1944,.
04,08/11/1950,12/31/2010
05,07/18/1941,12/12/2012
06,.,12/29/2011
07,.,12/31/2011
08,.,02/14/2012


### Match-Merging
* The program writes obdervations for matches only.
* The IN= data set option creates a variable that can be used to identify matches and non-matches.

When you combine two data sets, you can use IN= data set option to track which of the original data sets contributes to each observation in the new data set. 

The following code is equivalent to Inner Joins in PROC SQL.

In [19]:
*Ex2_match_merge_sql_outer.sas;
options nocenter nodate nonumber;
** DATA Step Merge (exact match);
data Exact_Match;
 merge  BIRTH (in=b) DEATH (in=d);
   by id;
 if b=d;
 run;
title1 'DATA Step Merge - Exact Match';
proc print data=Exact_Match noobs;
run;

id,dob,dod
04,08/11/1950,12/31/2010
05,07/18/1941,12/12/2012


### DATA Step Merge (Right Merge)

The following code is equivalent to Right Joins in PROC SQL.

In [25]:
*Ex2_match_merge_sql_outer.sas (Part 12);
options nocenter nodate nonumber;
** DATA Step Merge ;
data right_merge;
 merge  BIRTH DEATH (in=d); 
 by id;
 if d;
 run;
title1 'DATA Step Merge (Right Merge)';
proc print data=right_merge noobs;
run;

id,dob,dod
04,08/11/1950,12/31/2010
05,07/18/1941,12/12/2012
06,.,12/29/2011
07,.,12/31/2011
08,.,02/14/2012


In [27]:
*Ex2_match_merge_sql_outer.sas (Part 14);
options nocenter nodate nonumber;
*** DATA Step Merge (nonmatch in the RIGHT data set) vs. PROC SQL subquery;
data Not_in_death;
 merge  BIRTH(in=b) DEATH (in=d); 
 by id;
 if b=1 & d ne 1;;
 run;
title1 'DATA Step Merge - Finding BIRTH IDs that are not in the DEATH file';
proc print data=Not_in_death noobs;
run;

id,dob,dod
01,01/09/1954,.
02,09/12/1959,.
03,03/31/1944,.


In [28]:
*Ex2_match_merge_sql_outer.sas (Part 15);
options nocenter nodate nonumber;
*PROC SQL subquery finding BIRTH IDs that are not in the DEATH file; 
proc sql;
title1 'SQL subquery - Finding BIRTH IDs that are not in the DEATH file';
  select id, dob
  from birth
  where id not in(select id from death);
quit;


id,dob
01,01/09/1954
02,09/12/1959
03,03/31/1944


In [29]:
*Ex2_match_merge_sql_outer.sas (Part 16);
options nocenter nodate nonumber;
** DATA Step Merge (nonmatch in the RIGHT data set);
data Not_in_birth;
 merge  BIRTH(in=b) DEATH (in=d); 
 by id;
 if d=1 & b ne 1;;
 run;
title1 'DATA Step Merge - Finding DEATH IDs that are not in the BIRTH file';
proc print data=Not_in_birth noobs;
run;

id,dob,dod
06,.,12/29/2011
07,.,12/31/2011
08,.,02/14/2012


In [1]:
*Ex3_merge_update_BY.sas;
options nocenter nodate nonumber;
proc datasets kill nolist nodetails; quit;
data work.MASTER; 
infile datalines firstobs=2 truncover;
input pt_id $ 1-4 Name $ 6-13 @16 visit_date mmddyy10.;
FORMAT visit_date mmddyy10.;
datalines;
1234567890123456789012345
P001 Mary      07/23/2016
;
proc sort data=work.MASTER; by  pt_id; run;

data work.TRANS; 
infile datalines firstobs=2 truncover;
input pt_id $ 1-4 Name $ 6-13 @16 visit_date mmddyy10.;
FORMAT visit_date mmddyy10.;
datalines;
1234567890123456789012345
P001           09/24/2016
P001 Ann-Mary  11/30/2016
P001 Ann-Mary   
;

proc sort data=work.TRANS; by  pt_id; run;

title1 'LISTNG - WORK.MASTER';
proc print data=work.MASTER; 
run;

title1 'LISTNG - WORK.TRANS';
proc print data=work.TRANS; 
run;
title1;

SAS Connection established. Subprocess id is 2364



Obs,pt_id,Name,visit_date
1,P001,Mary,07/23/2016
Obs,pt_id,Name,visit_date
1,P001,,09/24/2016
2,P001,Ann-Mary,11/30/2016
3,P001,Ann-Mary,.


In [2]:
*Ex3_merge_update_BY.sas (Part 2);
options nocenter nodate nonumber;
data work.Merged_NEW;
 MERGE work.MASTER 
       work.TRANS; 
   by  pt_id;
run;
title1 'WORK.MERGED_NEW';
proc print data=
     work.Merged_NEW; 
run;

Obs,pt_id,Name,visit_date
1,P001,,09/24/2016
2,P001,Ann-Mary,11/30/2016
3,P001,Ann-Mary,.


### Update Statement
* can be used to update selected values of certain variables in a master file (e.g., MASTER) using a transaction file (e.g., TRANSECT below).

In [3]:
*Ex3_merge_update_BY.sas (Part 3);
options nocenter nodate nonumber;
data work.Updated_NEW;
 UPDATE work.MASTER (IN=O) 
        work.TRANS (IN=T);
  by  pt_id;
run;
title1 'WORK.Updated_NEW '; 
proc print data=
     work.Updated_NEW; 
run;

Obs,pt_id,Name,visit_date
1,P001,Ann-Mary,11/30/2016


###  Example Data Set

In [5]:
*Ex3_merge_update_BY.sas (Part 4) - Subway Master File;
DATA master;
INFILE DATALINES DLM=',';
INPUT Id item & $14.  sub_6_inch footlong;
DATALINES;
1,Cold Cut Combo,3.50, 5.00
2,Pizza Sub,3.50, 5.00
3,Spicy Italian,3.50, 5.00
4,Veggie Delite, 3.50, 5.00
5,Turkey Breast, 4.00, 6.00
6,Tuna,             4.00, 6.00
7,Veggie Patty,     4.00, 6.00
8,Subway Club,     4.50, 7.00
9,Subway Melt,     4.50, 7.00
10,Steak & Cheese, 4.50, 7.25
11,Roast Beef,     4.50, 7.25
;
PROC SORT DATA=master; 
  BY id; 
run;
title1 'Master File - Subway Menu';
Proc print data=master noobs; run;



Id,item,sub_6_inch,footlong
1,Cold Cut Combo,5.0,2.00
3,"Spicy Italian,",5.0,4.00
5,Turkey Breast,4.0,6.00
6,Tuna,4.0,6.00
7,Veggie Patty,4.0,6.00
8,Subway Club,4.5,7.00
9,Subway Melt,4.5,7.00
10,Steak & Cheese,4.5,7.25
11,Roast Beef,4.5,7.25


In [6]:
*Ex3_merge_update_BY.sas (Part 5) - Subway Transaction File;
DATA Transact;
INFILE DATALINES DLM=',';
INPUT Id item & $14.  sub_6_inch footlong;
DATALINES;
9,Subway Melt,     5.50, 8.00
10,Steak & Cheese, 5.50, 9.25
11,Roast Beef,     5.50, 8.25
;
PROC SORT DATA=Transact; 
  BY id; 
run;
title1 'Transaction File - Subway Menu';
Proc print data=Transact noobs; run;


Id,item,sub_6_inch,footlong
9,Subway Melt,5.5,8.00
10,Steak & Cheese,5.5,9.25
11,Roast Beef,5.5,8.25


### Update vs. Modify Statement

* UPDATE replaces an existing file with a new file, allowing you to add, delete, or rename columns.

* MODIFY performs an update in place by rewriting only those records that have changed, or by appending new records to the end of the file. 

In [7]:
*Ex3_merge_update_BY.sas (Part 6) - Subway Updated File;
DATA updated; 
 UPDATE master Transact; BY id;
run;
title 'Example of the UPDATE statement';
PROC PRINT DATA=updated noobs; 
run;
title;


Id,item,sub_6_inch,footlong
1,Cold Cut Combo,5.0,2.00
3,"Spicy Italian,",5.0,4.00
5,Turkey Breast,4.0,6.00
6,Tuna,4.0,6.00
7,Veggie Patty,4.0,6.00
8,Subway Club,4.5,7.00
9,Subway Melt,5.5,8.00
10,Steak & Cheese,5.5,9.25
11,Roast Beef,5.5,8.25


In [8]:
DATA master_x; 
 SET master;
run;
DATA master_x; 
 MODIFY master_x Transact; 
BY id;
run;
title 'Example of the MODIFY statement';
PROC PRINT DATA=Master_x noobs; 
run;
title;

Id,item,sub_6_inch,footlong
1,Cold Cut Combo,5.0,2.00
3,"Spicy Italian,",5.0,4.00
5,Turkey Breast,4.0,6.00
6,Tuna,4.0,6.00
7,Veggie Patty,4.0,6.00
8,Subway Club,4.5,7.00
9,Subway Melt,5.5,8.00
10,Steak & Cheese,5.5,9.25
11,Roast Beef,5.5,8.25


The following two code blocks (DATA Step and PROC SQL) 
    provide the same results -  there are other DATA step solutions
    (not shown here).

In [31]:
*Ex8_Sum_Retain_Do_Until.sas (Part 7);
options nocenter nodate nonumber;
*Count occurrences of dates by patient id;
data HAVE;
input pt_id $ vdate :$10.;
cards;
1      09/01/2017   
1      09/01/2017   
1      03/04/2018   
2      05/01/2017  
2      06/03/2017 
;
run;
proc sort data=HAVE;
by pt_id vdate;
run;

In [33]:
*Ex8_Sum_Retain_Do_Until.sas (Part 5);
options nocenter nodate nonumber;
** Summarize data using PROC SUMMARY;
proc summary data=have nway;
  var value;
  class id;  
  output out=want (drop =_type_ _freq_)
    sum=sum_value;
run;
title1 'Sumarizing by Group - using PROC SUMMARY';
proc print data=want noobs; run;

ID,sum_value
1,90
2,80
3,60


In [40]:
*Ex8B_Collapse_multi_records.sas (Part 1);
options nocenter nodate nonumber;
data have;
 input id type value count;
 cards;
 1 1 32 2
 1 1 10 7
 1 2 20 10
 1 2 59 2
 1 3 54 1
 1 3 82 4
 1 4 68 2
 1 5 56 0
 2 1 52 8
 2 5 64 9
 2 3 76 6
 2 4 98 8
 2 2 39 9
 2 4 96 5
 3 1 58 2
 3 2 63 6
 3 4 72 3
 3 5 99 4
 3 3 37 1
 3 1 66 0
 ;
 title1 'Example Data Set';
proc print noobs; run;

id,type,value,count
1,1,32,2
1,1,10,7
1,2,20,10
1,2,59,2
1,3,54,1
1,3,82,4
1,4,68,2
1,5,56,0
2,1,52,8
2,5,64,9


In [41]:
*Ex8B_Collapse_multi_records.sas (Part 2);
title1 'Agggregate the values of the numeric variable';
options nocenter nodate nonumber;
 proc summary data=have nway missing;
   class id type;
   var value count;
   output out=sums(drop=_:) sum=;
 proc print data=sums noobs; run;

id,type,value,count
1,1,42,9
1,2,79,12
1,3,136,5
1,4,68,2
1,5,56,0
2,1,52,8
2,2,39,9
2,3,76,6
2,4,194,13
2,5,64,9


In [42]:
 *Ex8B_Collapse_multi_records.sas (Part 3);
options nocenter nodate nonumber;
title1 'Transpose the Agggregated table';
 proc transpose data=sums out=trans
   prefix=type_;
   by id;
   id type;
 proc print data=trans noobs; run;

id,_NAME_,type_1,type_2,type_3,type_4,type_5
1,value,42,79,136,68,56
1,count,9,12,5,2,0
2,value,52,39,76,194,64
2,count,8,9,6,13,9
3,value,124,63,37,72,99
3,count,2,6,1,3,4


In [43]:
 *Ex8B_Collapse_multi_records.sas (Part 4);
options nocenter nodate nonumber;
title1 'Merge the Transposed data using two SET statements';
 data want;
   set trans(where=(_name_='count'));
   count=sum(of type_:);
   set trans(where=(_name_='value'));
   drop _name_;
 proc print data=want noobs; run;


id,type_1,type_2,type_3,type_4,type_5,count
1,42,79,136,68,56,28
2,52,39,76,194,64,45
3,124,63,37,72,99,16


In [15]:
*Ex10_first_var_last_var.sas (Part 1);
options nodate nocenter nonumber; 
DATA work.Have;
INPUT ID $ calorie_intake;
 DATALINES;
 A 200
 A 800
 A 500
 C 250
 C 850
 C 550
 B 300
 B 900
 B 600
 D 260
 D 900
 D 800
;
PROC SORT data=work.Have 
   out=work.Sorted_have; 
 BY ID; 
run;


In [16]:
*Ex10_first_var_last_var.sas (Part 2);
options nocenter nodate nonumber nosource;
ods html close;
DATA _NULL_;
 SET work.sorted_have; BY ID; 
 PUTLOG ID= First.ID=  LAST.ID= calorie_intake=;
run;

## Combine summary value (average) to the detail dataset and then calculate the deviation of the individual weight from the mean.

## Proc step/Data step approach 

In [3]:
proc means data=sashelp.class noprint;
var weight;
output out=summary_data_m (drop=_TYPE_ _FREQ_) mean=ave_weight;
run;

title1 'Summarized values from PROC MEANS output data set';
proc print data=summary_data_m noobs; run;

ave_weight
100.026


In [19]:
*Ex11_summary_detail.sas (Part 2);
options nocenter nodate nonumber;
*** Summary value using PROC SUMMARY;
proc summary data=sashelp.class;
     var weight;
      output out=summary_data_s (drop=_TYPE_ _FREQ_)
      mean(weight)=avg_weight;
run;
title1 'Summarized values from PROC SUMMARY output data set';
proc print data=summary_data_s noobs; 
run;

avg_weight
100.026


In [21]:
*Ex11_summary_detail.sas (Part 3);
options nocenter nodate nonumber;
data class;
        if _n_=1 then  set summary_data_m;
        set sashelp.class (keep=name weight);
        weight_deviation=1-(weight/avg_weight);
run;
title1 'Combine the summary data with the detail data using two SET statements';
proc print data=class;
var name weight avg_weight weight_deviation;
format avg_weight weight  5.1  weight_deviation percent8.2;
run;

Obs,Name,Weight,avg_weight,weight_deviation
1,Alfred,112.5,100.0,(12.47%)
2,Alice,84.0,100.0,16.02%
3,Barbara,98.0,100.0,2.03%
4,Carol,102.5,100.0,( 2.47%)
5,Henry,102.5,100.0,( 2.47%)
6,James,83.0,100.0,17.02%
7,Jane,84.5,100.0,15.52%
8,Janet,112.5,100.0,(12.47%)
9,Jeffrey,84.0,100.0,16.02%
10,John,99.5,100.0,0.53%


In [22]:
*Ex11_summary_detail.sas (Part 4);
title1 'Combine the summary data with the detail data using PROC SQL';
PROC SQL;
select  name 
       ,weight format=5.1
       ,mean(weight) as avg_weight format=5.1
       ,1-(weight/calculated avg_weight) 
          as weight_deviation format=percent8.2
  from sashelp.class;
 quit;


Name,Weight,avg_weight,weight_deviation
Alfred,112.5,100.0,(12.47%)
Alice,84.0,100.0,16.02%
Barbara,98.0,100.0,2.03%
Carol,102.5,100.0,( 2.47%)
Henry,102.5,100.0,( 2.47%)
James,83.0,100.0,17.02%
Jane,84.5,100.0,15.52%
Janet,112.5,100.0,(12.47%)
Jeffrey,84.0,100.0,16.02%
John,99.5,100.0,0.53%


In [23]:
*Ex11_summary_detail.sas (Part 5);
* DATA Step Approach  ;
data detail_class;
 length new_var $1;
 set sashelp.class;
  new_var='C';
 run;
 data x_summary_data_m;
   length new_var $1;
   set summary_data_m;
     new_var='C';
 run;

data mclass (drop=new_var);
 merge detail_class 
        x_summary_data_m;
 by new_var;
 weight_deviation=1-(weight/avg_weight);
 run;

 title1 'DATA step using the MERGE statement';
proc print data=mclass;
format weight avg_weight 5.1 
       weight_deviation percent8.2;
run;




Obs,Name,Sex,Age,Height,Weight,avg_weight,weight_deviation
1,Alfred,M,14,69.0,112.5,100.0,(12.47%)
2,Alice,F,13,56.5,84.0,100.0,16.02%
3,Barbara,F,13,65.3,98.0,100.0,2.03%
4,Carol,F,14,62.8,102.5,100.0,( 2.47%)
5,Henry,M,14,63.5,102.5,100.0,( 2.47%)
6,James,M,12,57.3,83.0,100.0,17.02%
7,Jane,F,12,59.8,84.5,100.0,15.52%
8,Janet,F,15,62.5,112.5,100.0,(12.47%)
9,Jeffrey,M,13,62.5,84.0,100.0,16.02%
10,John,M,12,59.0,99.5,100.0,0.53%


In [24]:
*Ex11_summary_detail.sas (Part 6);
*** PROC Step, CALL SYMPUTX, DATA Step;
Options nocenter nodate nonumber;
proc means data=sashelp.class noprint;
var weight;
output out=mystats mean=ave_weight;
run;

data _null_;
set mystats;
call symputx('AverageWeight',ave_weight);
run;

data x_class;
        set sashelp.class (keep=name weight);
        weight_deviation=1-(weight/&AverageWeight);
run;
title1 "PROC Step, CALL SymputX, and DATA Step";
title2 "Mean weight: %sysfunc(putn(&AverageWeight, 5.1)) lbs";
proc print data=x_class;
var name weight weight_deviation;
format weight 5.1  weight_deviation   percent8.2;
run;

Obs,Name,Weight,weight_deviation
1,Alfred,112.5,(12.47%)
2,Alice,84.0,16.02%
3,Barbara,98.0,2.03%
4,Carol,102.5,( 2.47%)
5,Henry,102.5,( 2.47%)
6,James,83.0,17.02%
7,Jane,84.5,15.52%
8,Janet,112.5,(12.47%)
9,Jeffrey,84.0,16.02%
10,John,99.5,0.53%


In [27]:
*Ex12_SUM_Statement_vs_2_SETs.sas (Part 1);
options nocenter nodate nonumber;
DATA sale_by_mon ;
  INPUT mon $  sale @@;
  cum_sale+sale;
  DATALINES;
  Jan 164083 Feb 164260 Mar 163747 Apr 164759 
  May 165617 Jun 166098 Jul 167305 Aug 167797 
  Sep 169407 Oct 170681 Nov 171025 Dec 172995
  ;
run;
title1 'Example Data Set';
PROC PRINT DATA=sale_by_mon ; 
  FORMAT sale cum_sale dollar12.;
   SUM sale ;
  run;

Obs,mon,sale,cum_sale
1,Jan,"$164,083","$164,083"
2,Feb,"$164,260","$328,343"
3,Mar,"$163,747","$492,090"
4,Apr,"$164,759","$656,849"
5,May,"$165,617","$822,466"
6,Jun,"$166,098","$988,564"
7,Jul,"$167,305","$1,155,869"
8,Aug,"$167,797","$1,323,666"
9,Sep,"$169,407","$1,493,073"
10,Oct,"$170,681","$1,663,754"


In [30]:
*Ex12_SUM_Statement_vs_2_SETs.sas (Part 2);
DATA xsale;
  SET sale_by_mon(keep=cum_sale)
       POINT=last nobs=last;
  SET sale_by_mon (drop=cum_sale);
  Percent_sale = sale/cum_sale;
run;
title1 'Combining the summary data with the detail data';
PROC PRINT DATA=xsale; 
  VAR mon sale Percent_sale;
  SUM sale Percent_sale;
  FORMAT sale dollar10. Percent_sale percent12.2;
run;


Obs,mon,sale,Percent_sale
1,Jan,"$164,083",8.17%
2,Feb,"$164,260",8.18%
3,Mar,"$163,747",8.16%
4,Apr,"$164,759",8.21%
5,May,"$165,617",8.25%
6,Jun,"$166,098",8.27%
7,Jul,"$167,305",8.33%
8,Aug,"$167,797",8.36%
9,Sep,"$169,407",8.44%
10,Oct,"$170,681",8.50%
